In [1]:
import pandas as pd
import numpy as np

from graphframes import *
from pyspark import SparkContext

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from py2neo import Graph
ip = "192.168.0.103"
graph = Graph("bolt://%s:7687"%ip, password="123456")

##### Degree Table

In [3]:
query = """
CALL algo.closeness.stream("Person", "KNOWS", {improved: true})
YIELD nodeId, centrality
RETURN algo.getNodeById(nodeId).name as name, centrality
ORDER BY centrality DESC
"""

df_rank = graph.run(query).to_data_frame()
df_rank = spark.createDataFrame(df_rank.values.tolist(), list(df_rank.columns.values))
df_rank.createOrReplaceTempView('df_rank')
df_rank.show(5)

+-------+-------------------+
|   name|         centrality|
+-------+-------------------+
| Gerald| 0.1595732606968562|
|  Lenny| 0.1560661560661561|
| Rainey| 0.1560661560661561|
|Sparrow|0.15270989464537854|
|   Desi|0.14793771043771045|
+-------+-------------------+
only showing top 5 rows



##### Component Table

In [4]:
query = """
CALL algo.unionFind.stream("Person", "KNOWS")
YIELD nodeId, setId
RETURN algo.getNodeById(nodeId).name as name, setId as component
"""

graph.run(query).to_data_frame()

df_tmp = graph.run(query).to_data_frame()
df_tmp = spark.createDataFrame(df_tmp.values.tolist(), list(df_tmp.columns.values))
df_tmp.createOrReplaceTempView('df_tmp')
df_tmp.show(5)

+--------+---------+
|    name|component|
+--------+---------+
|     Roy|        0|
|  Philip|        1|
|    Joey|        1|
|Isabella|        1|
|  Howard|        1|
+--------+---------+
only showing top 5 rows



##### Size Table (for filtering)

In [5]:
sql = """
select component, count(*) as sz
from df_tmp
group by component
order by sz desc
"""

df_sz = spark.sql(sql)
df_sz.createOrReplaceTempView('df_sz')
df_sz.show(5)

+---------+---+
|component| sz|
+---------+---+
|       31| 39|
|        1| 20|
|       16| 12|
|       35|  8|
|       43|  5|
+---------+---+
only showing top 5 rows



#### Mark Core


<img src="assets/50_closeness_center.svg" alt="Drawing" style="width: 800px;"/>

In [6]:
sql = """
with numbered as (
select
  a.name, a.component, b.centrality
  ,row_number() over (partition by component order by centrality desc, a.name) as rn
from df_tmp as a
join df_rank as b
on a.name = b.name
)
select a.name, a.component, a.centrality, b.sz
from numbered a
join df_sz as b
on a.component = b.component
where rn = 1
  and b.sz > 2
"""
df_core = spark.sql(sql).toPandas()
df_core.head()

,name,component,centrality,sz
0,Joanna,43,0.030303,5
1,Gerald,31,0.159573,39
2,Helga,1,0.078512,20
3,Cornelia,35,0.035354,8
4,Meg,16,0.048309,12


In [7]:
# Unmark Core
cql = """
MATCH (n)
REMOVE n:Core
return n
"""
graph.run(cql)

In [8]:
cql = """
MATCH (n)
WHERE n.name IN [%s]
SET n:Core
return n
"""%", ".join("\"%s\""%core for core in df_core.name.values)
print(cql)
graph.run(cql)


MATCH (n)
WHERE n.name IN ["Joanna", "Gerald", "Helga", "Cornelia", "Meg"]
SET n:Core
return n

